In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df=pd.read_csv("heart.csv")

In [3]:
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


# outlier removal

In [4]:
numerical_features = [feature for feature in df.columns if df[feature].dtype=="int"]

In [5]:
def outlier_detection(column, df):
    outlier=[]
    for x in  df[column]:
        z=np.divide(x-np.mean(df[column]),np.std(df[column]))
        if z > 3:
            outlier.append(x)
            df[column].replace(x,math.ceil(np.mean(df[column])),inplace=True)
    return outlier     
    

for column in numerical_features:
    outlier_detection(column, df)

In [6]:
outlier_detection("RestingBP", df)

[185]

# Droping any duplicate

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.shape

(918, 12)

In [9]:
df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [10]:
categorical_features = [feature for feature in df.columns if df[feature].dtype=="object"]

In [11]:
categorical_features

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

# converting categorical features to one_hot code

In [12]:
Sex=pd.get_dummies(df["Sex"])
ChestPainType =pd.get_dummies(df["ChestPainType"])
ExerciseAngina= pd.get_dummies(df["ExerciseAngina"])
RestingECG = pd.get_dummies(df["RestingECG"])
ST_Slope =  pd.get_dummies(df["ST_Slope"])

data1=pd.concat([Sex,ChestPainType, RestingECG, ExerciseAngina, ST_Slope], axis=1)

In [13]:
data1

,F,M,ASY,ATA,NAP,TA,LVH,Normal,ST,N,Y,Down,Flat,Up
0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,0,1,0,0,1,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,0,1,0,0,0,1,0,1,0,1,0,0,1,0
914,0,1,1,0,0,0,0,1,0,1,0,0,1,0
915,0,1,1,0,0,0,0,1,0,0,1,0,1,0
916,1,0,0,1,0,0,1,0,0,1,0,0,1,0


In [14]:
data2=df[numerical_features].iloc[:,:-1].copy()

In [15]:
X=pd.concat([data1,data2], axis=1)
y=df.iloc[:,-1]

# scaling the features

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
scaler=StandardScaler()
X=scaler.fit_transform(X)


# Dimensionality reduction

In [18]:
X.shape

(918, 19)

In [19]:
from sklearn.decomposition import PCA

In [20]:
pca=PCA(13)
X_pca=pca.fit_transform(X)

In [21]:
X_pca.shape

(918, 13)

In [22]:
pca.explained_variance_ratio_

array([0.221076  , 0.10486961, 0.09550329, 0.07534035, 0.07464611,
       0.06553906, 0.06362236, 0.05915402, 0.05191146, 0.04727373,
       0.04319409, 0.04061528, 0.03196244])

In [23]:
X=pd.DataFrame(X_pca)

X.to_csv("X")

In [24]:
y.to_csv("y")